In [20]:
import sys
import torch
import pandas as pd
#import matplotlib.pyplot as plt
import pickle
import numpy as np
from multiprocessing import Pool

'''
print('loading data...')
train = pd.read_csv('./data/train.csv')

print('loading model...')
path = './models/latest2.pt'
checkpoint = torch.load(path, map_location='cpu')
print('done!')

model = checkpoint['rnn']
embed = checkpoint['embed']
'''

def format_post(row):
    """Takes a row and formats it in a consistent manner
    for training and feature extraction."""
    #all fields denoted by newline
    result = '\n'
    result += row.region + '\n'
    result += row.city + '\n'
    result += row.parent_category_name + '\n'
    result += row.category_name + '\n'
    #parameters are seperated by tab bytes
    if not pd.isnull(row.param_1):
        result += str(row.param_1) + '\t'
    if not pd.isnull(row.param_2):
        result += str(row.param_2) + '\t'
    if not pd.isnull(row.param_3):
        result += str(row.param_3) + '\t'
    result += '\n'
    result += str(row.title)
    description = row.description
    if type(description) != type(str):
        description = str(description)
    #remove newlines from description body
    description = description.replace('\n', ' ')
    result += description
    return result

def get_state(text, model=model, embed=embed):
    """Gets hidden state features for model"""
    state = model.state0(batch_size=1)
    encoded = text.encode('utf-8')
    for byt in encoded:
        emb = embed(torch.tensor(byt))
        state, output = model(emb, state)
    return state[1].detach().numpy()[0, 0]

def state_at_index(idx, data):
    """Takes an index and dataframe and returns an index
    and extracted internal state.
    """
    row = data.loc[idx]
    post = format_post(row)
    state = get_state(post)
    return idx, state

def multi_extract(n_threads=1, data=train, save_path='fts.pkl'):
    result = dict()
    indices = list(data.index)
    threads = []
    with Pool(processes=n_threads) as pool:
        for n in range(n_threads):
            idx = indices.pop(0)
            threads.append(pool.apply_async(state_at_index, (idx, data)))
        while True:
            threads.append(pool.apply_async(state_at_index, (idx, data)))
            idx, state = results.pop(0).get()
            result[idx] = state
            if len(result) % 100 == 0:
                print(len(result), 'states extracted. Saving to {}...'.format(save_path))
                with open(save_path, 'wb') as f:
                    pickle.write(result, f)
                    print('done!')
            print('got index: ', idx)
            print('state length: ', len(state))

1503424
0
1503423
1
1503422


In [6]:
path = './data/few_fts.npy'
try:
    states = np.load(path)
except FileNotFoundError:
    states = None
for i, row in train.iterrows():
    if type(states) == type(None):
        desc = format_post(row)
        states = get_state(model, embed, desc).reshape(1, -1)
        print(i, ' done.')
    elif i not in range(states.shape[0]):
        desc = format_post(row)
        state = get_state(model, embed, desc).reshape(1, -1)
        state = state.reshape(1, -1)
        states = np.concatenate([states, state], axis=0)
        print(i, ' done.')
        if i % 10 == 0:
            print('saving...')
            np.save(path, states)

41  done.


KeyboardInterrupt: 

In [ ]:
states

In [ ]:
states_short.shape